Paso 1: Cargar el dataset con augmentación (TensorFlow)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

data_dir = "/home/alvaro/Escritorio/prediccion_cancer/colon_image_sets"
img_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=preprocess_input
)

generator = datagen.flow_from_directory(
    directory=data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False
)


2025-04-20 12:45:06.589952: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-20 12:45:06.590332: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-20 12:45:06.592570: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-20 12:45:06.598653: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745145906.609286   57072 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745145906.61

Found 10000 images belonging to 2 classes.


In [ ]:
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Model

# Cargar modelo preentrenado sin la capa final
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, pooling='avg', weights='imagenet')
feature_model = Model(inputs=base_model.input, outputs=base_model.output)

# Extraer features
features_list = []
labels_list = []

for i in range(len(generator)):
    x_batch, y_batch = generator[i]
    f_batch = feature_model.predict(x_batch)
    features_list.append(f_batch)
    labels_list.append(y_batch)
    
    if (i + 1) * batch_size >= generator.samples:
        break

X = np.concatenate(features_list)
y = np.concatenate(labels_list)


2025-04-20 12:45:08.065648: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


NameError: name 'generator' is not defined